# Keyboard Hints

> Components for displaying keyboard shortcut hints to users.

In [ ]:
#| default_exp components.hints

In [ ]:
#| export
from __future__ import annotations
from collections import defaultdict
from fasthtml.common import Div, Span

from cjm_fasthtml_keyboard_navigation.core.actions import KeyAction
from cjm_fasthtml_keyboard_navigation.core.manager import ZoneManager

from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_styles
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_wrap, gap, items
)
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_family, font_weight
from cjm_fasthtml_tailwind.utilities.spacing import m
from cjm_fasthtml_tailwind.core.base import combine_classes

## Single Hint Badge

In [ ]:
#| export
def render_hint_badge(
    key_display: str,       # formatted key (e.g., "↑/↓", "Space")
    description: str,       # action description
    style: str = "ghost"    # badge style (ghost, outline, soft, dash)
) -> Div:                   # hint badge component
    """Render a single keyboard hint as a badge."""
    badge_style = getattr(badge_styles, style, badge_styles.ghost)
    
    return Div(
        Span(key_display, cls=combine_classes(font_family.mono, font_weight.bold)),
        Span(description),
        cls=combine_classes(badge, badge_style, gap(1))
    )

In [ ]:
# Test hint badge
from fasthtml.common import to_xml

hint_badge = render_hint_badge("↑/↓", "Navigate")
html = to_xml(hint_badge)
assert "↑/↓" in html
assert "Navigate" in html
assert "badge" in html

## Hint Group

In [ ]:
#| export
def render_hint_group(
    group_name: str,         # group header text
    hints: list[tuple[str, str]],  # list of (key_display, description) tuples
    badge_style: str = "ghost"     # badge style for this group
) -> Div:                    # group container with header and hints
    """Render a group of related keyboard hints."""
    badges = [render_hint_badge(key, desc, badge_style) for key, desc in hints]
    
    return Div(
        Span(
            group_name,
            cls=combine_classes(font_size.xs, font_weight.semibold, m.r(2))
        ),
        *badges,
        cls=combine_classes(flex_display, items.center, gap(1), flex_wrap.wrap)
    )

In [ ]:
# Test hint group
group = render_hint_group(
    "Navigation",
    [("↑/↓", "Move"), ("←/→", "Switch")]
)
html = to_xml(group)
assert "Navigation" in html
assert "Move" in html
assert "Switch" in html

## Hints from Actions

In [ ]:
#| export
def group_actions_by_hint_group(
    actions: tuple[KeyAction, ...]  # actions to group
) -> dict[str, list[KeyAction]]:    # grouped actions
    """Group actions by their hint_group attribute."""
    groups = defaultdict(list)
    for action in actions:
        if action.show_in_hints and action.description:
            groups[action.hint_group].append(action)
    return dict(groups)

In [ ]:
#| export
def render_hints_from_actions(
    actions: tuple[KeyAction, ...],  # actions to display hints for
    badge_style: str = "ghost"       # badge style
) -> Div:                            # container with all hint groups
    """Render keyboard hints from action configurations."""
    groups = group_actions_by_hint_group(actions)
    
    group_components = []
    for group_name, group_actions in groups.items():
        hints = [(a.get_display_key(), a.description) for a in group_actions]
        group_components.append(
            render_hint_group(group_name, hints, badge_style)
        )
    
    return Div(
        *group_components,
        cls=combine_classes(flex_display, flex_wrap.wrap, gap(4))
    )

In [ ]:
# Test hints from actions
actions = (
    KeyAction(key=" ", htmx_trigger="x", description="Select", hint_group="Selection"),
    KeyAction(key="Delete", htmx_trigger="y", description="Remove", hint_group="Actions"),
    KeyAction(key="Enter", js_callback="z", description="Open", hint_group="Actions"),
    KeyAction(key="Backspace", htmx_trigger="w", description="Delete", show_in_hints=False),  # Hidden
)

hints_component = render_hints_from_actions(actions)
html = to_xml(hints_component)
assert "Selection" in html
assert "Actions" in html
assert "Select" in html
assert "Remove" in html
assert "Delete" not in html  # show_in_hints=False

## Full Keyboard Hints

In [ ]:
#| export
def render_keyboard_hints(
    manager: ZoneManager,                      # the zone manager
    include_navigation: bool = True,           # include navigation hints
    include_zone_switch: bool = True,          # include zone switching hints
    badge_style: str = "ghost",                # badge style
    container_id: str = "kb-hints"             # container element ID
) -> Div:                                      # complete hints component
    """Render complete keyboard hints for a zone manager."""
    from cjm_fasthtml_keyboard_navigation.core.key_mapping import format_key_for_display
    
    sections = []
    
    # Navigation hints
    if include_navigation:
        nav_hints = [("↑/↓", "Navigate")]
        sections.append(render_hint_group("Navigation", nav_hints, badge_style))
    
    # Zone switching hints
    if include_zone_switch and len(manager.zones) > 1:
        prev_key = format_key_for_display(manager.prev_zone_key)
        next_key = format_key_for_display(manager.next_zone_key)
        switch_hints = [(f"{prev_key}/{next_key}", "Switch Panel")]
        sections.append(render_hint_group("Panels", switch_hints, badge_style))
    
    # Action hints (grouped by hint_group)
    action_hints = render_hints_from_actions(manager.actions, badge_style)
    sections.append(action_hints)
    
    return Div(
        *sections,
        id=container_id,
        cls=combine_classes(flex_display, flex_wrap.wrap, gap(4), font_size.sm)
    )

In [ ]:
# Test full keyboard hints
from cjm_fasthtml_keyboard_navigation.core.focus_zone import FocusZone

zone1 = FocusZone(id="z1")
zone2 = FocusZone(id="z2")

manager = ZoneManager(
    zones=(zone1, zone2),
    actions=(
        KeyAction(key=" ", htmx_trigger="toggle", description="Select", hint_group="Selection"),
    )
)

hints = render_keyboard_hints(manager)
html = to_xml(hints)

assert 'id="kb-hints"' in html
assert "Navigate" in html
assert "Switch Panel" in html  # Two zones = show zone switch
assert "Select" in html

In [ ]:
# Single zone = no zone switch hint
single_manager = ZoneManager(zones=(zone1,), actions=())
hints = render_keyboard_hints(single_manager)
html = to_xml(hints)
assert "Switch Panel" not in html

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()